In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

SRC = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_시간대별_정류장_노선_추정승하차+운행횟수+순서+링크거리_보정.csv"
# 출력 파일(필요 시 경로만 바꾸세요)
DST = "/Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_링크_구간거리_from_보정.csv"

# (선택) 특정 노선만 필터하고 싶으면 사용
ROUTE_ID = 100100037     # 노선번호 172의 노선ID (파일에 보이는 값)
USE_ROUTE_FILTER = True  # 여러 노선 섞인 파일이면 True

df = pd.read_csv(SRC, encoding="utf-8-sig")

if USE_ROUTE_FILTER:
    df = df[df["노선ID"] == ROUTE_ID].copy()

# 시간/날짜 중복이 있으니 정류장_순서별 평균(또는 최빈/첫값)으로 집계
links = (
    df.groupby("정류장_순서", as_index=False)["링크_구간거리(m)"]
      .mean()
      .sort_values("정류장_순서")
)

# 품질 보정: 1번 정류장은 0m 고정, 음수/결측 방지
links.loc[links["정류장_순서"] == 1, "링크_구간거리(m)"] = 0.0
links["링크_구간거리(m)"] = links["링크_구간거리(m)"].clip(lower=0)
if links["링크_구간거리(m)"].isna().any():
    fill_val = links.loc[links["정류장_순서"] != 1, "링크_구간거리(m)"].median()
    links["링크_구간거리(m)"] = links["링크_구간거리(m)"].fillna(fill_val)

# 정밀도 정리(원하면 반올림)
links["링크_구간거리(m)"] = links["링크_구간거리(m)"].round(1)

# (선택) 점검용 누적거리
links["누적_거리(m)"] = links["링크_구간거리(m)"].cumsum()

# 파이프라인 호환: 최소 컬럼만 저장
links_out = links[["정류장_순서", "링크_구간거리(m)"]]
Path(DST).parent.mkdir(parents=True, exist_ok=True)
links_out.to_csv(DST, index=False, encoding="utf-8-sig")

print("✅ 저장:", DST)
print(links_out.head())

✅ 저장: /Users/minwoo/Desktop/softeer/data_engineering_course_materials/missions/final_project/data/172_링크_구간거리_from_보정.csv
   정류장_순서  링크_구간거리(m)
0       1         0.0
1       2       128.0
2       3       314.0
3       4       234.0
4       5       336.0
